In [104]:
import torch
from model.model1 import Net
import pandas as pd
import random
import copy

In [105]:
# Load Training dataset
df = pd.read_csv('data/cleaned_train.csv')
df = df.sample(frac=1).reset_index(drop=True)

In [106]:
# Divide between train and test data
train = df[:800]
test = df[800:]
# Divide between x,y 
train_x = torch.tensor(train.iloc[:,3:].values).float()
train_y = torch.tensor(train.iloc[:,2].values).float()
test_x = torch.tensor(test.iloc[:,3:].values).float()
test_y = torch.tensor(test.iloc[:,2].values).float()


In [107]:
# Create the network
net = Net(inputs=17,outputs=1,dropout_prob=0)
# Move all to gpu
if torch.cuda.is_available():
   net = net.to("cuda:0")
   train_x = train_x.to("cuda:0")
   train_y = train_y.to("cuda:0")
   test_x = test_x.to("cuda:0")
   test_y = test_y.to("cuda:0")



In [108]:
# Prepare the network for training
epochs = 200
batch_size = 10
# Initialize loss function
loss_fn = torch.nn.BCELoss()
# Initialize optimizer
optm = torch.optim.Adam(params=net.parameters(),lr=1e-2)
scheduler1 = torch.optim.lr_scheduler.LinearLR(optm,1,0.1,epochs)


In [109]:
# Divide tensors into batch sizes
train_x = train_x.split(batch_size);
train_y = train_y.split(batch_size);

In [110]:
losses = []
accuracies = []
num_batches = 50
test_y = test_y.reshape((test_y.shape[0],1))
max_accuracy = 0
best_model = copy.deepcopy(net.state_dict())
for epoch in range(epochs):
    net.train()
    for i in range(num_batches):
        indx = random.randint(0,len(train_x)-1)
        x = train_x[indx]
        y = train_y[indx]
        net_out = net.forward(x,x.size()[0])
        #print(all(i == net_out[0] for i in net_out))
        loss = loss_fn.forward(net_out,y.reshape(y.size()[0],1))
        optm.zero_grad()
        loss.backward()
        optm.step()
    scheduler1.step()
    with torch.no_grad():
        net.eval()
        net_out = net.forward(test_x,test_x.size()[0])
        loss = loss_fn.forward(net_out,test_y.reshape(test_y.size()[0],1))
        predictions = net_out.round()
        accuracy = 1- (predictions - test_y).abs().sum()/test_y.size()[0]
        if accuracy > max_accuracy:
            best_model = copy.deepcopy(net.state_dict())
            max_accuracy = accuracy
        losses.append(loss)
        accuracies.append(accuracies)
        print(f'EPOCH: {epoch}, Loss: {loss}, Accuracy: {accuracy}')


EPOCH: 0, Loss: 0.4581195116043091, Accuracy: 0.791208803653717
EPOCH: 1, Loss: 0.42024660110473633, Accuracy: 0.8461538553237915
EPOCH: 2, Loss: 0.4115566313266754, Accuracy: 0.8241758346557617
EPOCH: 3, Loss: 0.4055011570453644, Accuracy: 0.8351648449897766
EPOCH: 4, Loss: 0.39766091108322144, Accuracy: 0.8351648449897766
EPOCH: 5, Loss: 0.3914257287979126, Accuracy: 0.8461538553237915
EPOCH: 6, Loss: 0.3918834328651428, Accuracy: 0.8681318759918213
EPOCH: 7, Loss: 0.4081686735153198, Accuracy: 0.8241758346557617
EPOCH: 8, Loss: 0.38242626190185547, Accuracy: 0.8351648449897766
EPOCH: 9, Loss: 0.37289994955062866, Accuracy: 0.8351648449897766
EPOCH: 10, Loss: 0.35680291056632996, Accuracy: 0.8461538553237915
EPOCH: 11, Loss: 0.35749825835227966, Accuracy: 0.8351648449897766
EPOCH: 12, Loss: 0.3691140115261078, Accuracy: 0.8351648449897766
EPOCH: 13, Loss: 0.3724166750907898, Accuracy: 0.8571428656578064
EPOCH: 14, Loss: 0.3726233243942261, Accuracy: 0.8351648449897766
EPOCH: 15, Loss

In [111]:
torch.save(best_model,'saved_models/best_model.pt')